## Set the environment

In [1]:
import cobra
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import flux_variability_analysis
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import escher
from escher import Builder
from utils import show_map
from utils.check_precursor_problem import check_precursor_problem

## Read the excel that have all gene reaction rule information

In [2]:
df_gene_reaction_rule =pd.read_excel('../data/gene reaction rule.xlsx')
# set the reaction id column as the index in order to map it to the model 
df_gene_reaction_rule = df_gene_reaction_rule.set_index(["reaction_id"],)

## Read the raw transcriptomics of temperature treatment  70 and 80 ℃

In [3]:
xl = pd.ExcelFile('../data/80vs70 ArrayData.xlsx') # read an excel file
names1 = xl.sheet_names  # write all the sheet names in one list

### Format all the data

In [5]:
def get_overall_data (names1):
    df_overall_data = xl.parse("Array_Annotation") # write sheet named "Array_Annotation" in excel into a dataframe
    df_overall_data = df_overall_data.drop(labels=[0,1,2,3,4],axis = 0) # drop the first 5 lines
    df_overall_data.columns = df_overall_data.iloc[0] # set the first line as the column names
    df_overall_data = df_overall_data.drop(labels=[5],axis = 0) # delete the line with label 5
    df_overall_data = df_overall_data.set_index(["UID"],) # set the index
    df_overall_data["SSO number"] = "" # add a new blank column with name 'SSO number'
    for  UID in df_overall_data.index:
        if "SSO" in df_overall_data["ID"].loc[UID]:
            df_overall_data["SSO number"].at[UID] = str(df_overall_data["ID"].loc[UID]).split("_")[0]+"_" # the reason for adding "_" is because some SSO numbers 
            #only has 4 numbers and others has 5 numbers, to avoide the conditions like this (2133 and 21338)
    for name in names1:
        if name not in ["Array_Annotation", "Metadata"]:# the sheet name in the list cannot be processed by the following codes
            df = xl.parse(name)
            df  = df.drop(labels=[0,1,2,3,4,5],axis = 0)
            df.columns = df.iloc[0]
            df = df.drop(labels=[6],axis = 0)
            df = df.set_index(["UID"],)
            column_name_70 = name + " 70" #set the column name to sheet name add temperature
            column_name_80 = name + " 80"
            df_overall_data[column_name_70] = ""
            df_overall_data[column_name_80] = "" # set empty column

            for i in set(df_overall_data.index).intersection(set(df.index)): # find the same items in two set

                df_overall_data[column_name_70].at[i] = df["70°C"].loc[i]
                df_overall_data[column_name_80].at[i] = df["80°C"].loc[i]
        
    return (df_overall_data)
df_overall_data = get_overall_data(names1)
df_overall_data = df_overall_data.set_index(["SSO number"],)
df_overall_data.head()

5,R,C,Name,ID,Array_67 70,Array_67 80,Array_68 70,Array_68 80,Array_86 70,Array_86 80,Array_85 70,Array_85 80
SSO number,,,,,,,,,,,,
,1,1,NaN,gi|16206:1-933_A.thaliana_Cab_mRNA_oligo1_553-608,116020,85320,118800,124647,86190,145044,163856,89737
,1,2,NaN,gi|16206:1-933_A.thaliana_Cab_mRNA_oligo1_553-608,119282,96720,129168,127391,99104,157584,119132,87409
,1,3,NaN,gi|16470|emb|X14212.1|_A.thaliana_RCA_mRNA_oligo2_1232-1288,,,,,65521,149152,,
,1,4,NaN,gi|16470|emb|X14212.1|_A.thaliana_RCA_mRNA_oligo2_1232-1288,,,,,,,,
,1,5,NaN,gi|1928871|gb|U91966.1|ATU91966_A.thaliana_rbcL_mRNA_oligo3_1088-1155,,,,,69295,155472,162288,59350


### Match transcriptome to the reaction id

In [6]:
def match_data_to_reaction_id(columns): 
    
    for column_name in columns:
        if "Array" in column_name: # select the column with "Array" in its name
            
            df_gene_reaction_rule[column_name] = ""
            # the following list is special because its items' reaction rule is special with both "and" and "or" in it
            list_id_with_special_gene_reaction_rule = ["carb_acetylcoa_1.2.7.1_PYRUFLAVREDUCT__RXN","oxp_redox_1.6.5.3_RXN0__5330",'carb_degra__gly_1.8.1.4_1.4.4.2_2.1.2.10_GCVMULTI__RXN']
            for rxn in df_gene_reaction_rule.index:
                
                if rxn not in list_id_with_special_gene_reaction_rule:
                    if 'spontaneous' not in str(df_gene_reaction_rule['genes'].loc[rxn]):
                        c = str(df_gene_reaction_rule["genes"].loc[rxn]).split(",")
                       
                        if len(c) > 1:# select the reactions catalyzed by enzymes with more subunits or isoenzymes

                            gene_numbers1 = []
                            
                            if "or" in str(df_gene_reaction_rule['gene_reaction_rule'].loc[rxn]):

                                for i in range(len(c)):
                                    e = "SSO" + c[i]+"_"

                                    if e in df_overall_data.index:
                                        
                                        if df_overall_data[column_name].loc[e] !='':
                                            gene_numbers1.append(df_overall_data[column_name].loc[e])
     

                            df_gene_reaction_rule[column_name].loc[rxn] = sum(gene_numbers1)
                    

                            if "and" in str(df_gene_reaction_rule['gene_reaction_rule'].loc[rxn]):               

                                for i in range(len(c)):
                                    e = "SSO" + c[i]+"_"

                                    if e in df_overall_data.index:

                                        gene_numbers1.append(df_overall_data[column_name].loc[e])

                                       
                            if '' in gene_numbers1:
                                df_gene_reaction_rule[column_name].loc[rxn] = ""
                             
                            else:
                                df_gene_reaction_rule[column_name].loc[rxn] = min(gene_numbers1)
                             

                        else:
                            for i in range(len(c)):
                                e = "SSO" + c[i]+"_"

                                if e in df_overall_data.index:
                                    df_gene_reaction_rule[column_name].loc[rxn] = df_overall_data[column_name].loc[e]  
                                    df_gene_reaction_rule[column_name].loc[rxn] = df_overall_data[column_name].loc[e]
                    else:
                        df_gene_reaction_rule[column_name].loc[rxn] = 1000000 # if spontaneous in it, set the number to 1000000.
                        df_gene_reaction_rule[column_name].loc[rxn] = 1000000
                elif rxn in ["oxp_redox_1.6.5.3_RXN0__5330","carb_acetylcoa_1.2.7.1_PYRUFLAVREDUCT__RXN"]:
                    g = df_gene_reaction_rule["genes"].loc[rxn].split("or")
                   
                    gene_numbers1 = []
                    
                    c1 = str(g[0]).split(",")
                    for i in range(len(c1)):

                        e = "SSO" + c1[i]+ "_"

                        if e in df_overall_data.index:
                            
                            gene_numbers1.append(df_overall_data[column_name].loc[e])
                                


                    gene_numbers3 = []
                   
                    c2 = str(g[1]).split(",")
                    for i in range(len(c2)):

                        e = "SSO" + c2[i]+"_"
                        if e in df_overall_data.index:
                        
                            gene_numbers3.append(df_overall_data[column_name].loc[e])


                    if '' not in gene_numbers3:
                        if '' in gene_numbers1:
                            df_gene_reaction_rule[column_name].loc[rxn] = min(gene_numbers3)
                        else:
                            df_gene_reaction_rule[column_name].loc[rxn] = min(gene_numbers3) + min(gene_numbers3)
                    else:
                        
                        if '' in gene_numbers1:
                            df_gene_reaction_rule[column_name].loc[rxn] = ''
                        else:
                            df_gene_reaction_rule[column_name].loc[rxn] = min(gene_numbers1)
                    
                    
                elif rxn == "carb_degra__gly_1.8.1.4_1.4.4.2_2.1.2.10_GCVMULTI__RXN":
                    g = str(df_gene_reaction_rule["genes"].loc[rxn]).split(")")
                    
                    gene_numbers1 = []
                    
                    c1 = str(g[0]).split(",")
                    c1[0] = c1[0][1:]

                    for i in range(len(c1)):

                        e = "SSO" + c1[i]+ "_"

                        if e in df_overall_data.index:

                            if df_overall_data[column_name].loc[e] !='':
                                gene_numbers1.append(df_overall_data[column_name].loc[e])
                            
    
                    gene_numbers3 = []
                    
                    c2 = str(g[1]).split(",")
                    c2[0] = c2[0][1:]
                    for i in range(len(c2)):

                        e = "SSO" + c2[i]+"_"
                        if e in df_overall_data.index:

                            gene_numbers3.append(df_overall_data[column_name].loc[e])
                          

                    
                    
                    if '' in gene_numbers3:
                         df_gene_reaction_rule.loc[rxn,column_name] = ''
                    else:
                        score = min(gene_numbers3)
                        df_gene_reaction_rule.loc[rxn,column_name] = min(sum(gene_numbers1),score)

    return (df_gene_reaction_rule)
list_columns = df_overall_data.columns
df_match_data_to_reaction_id = match_data_to_reaction_id(list_columns)
df_match_data_to_reaction_id.head()

C:\Users\yzhang2\Anaconda3\envs\archeapy36\lib\site-packages\pandas\core\indexing.py:205 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Reaction name,pathway,genes,gene_reaction_rule,Unnamed: 5,Unnamed: 6,Array_67 70,Array_67 80,Array_68 70,Array_68 80,Array_86 70,Array_86 80,Array_85 70,Array_85 80
reaction_id,,,,,,,,,,,,,,
T_ABC__transporter_beta__D__glucose,ABC transport,NaN,"2847,3066,2848,3067,2849,3068,2850,3069",and,NaN,NaN,142229,63180,,,196101,99840,215040,141324
carb_entner_1.1.1.359_GLUCOSE__1__DEHYDROGENASE__NAD+__RXN,GDH,NaN,"3003,3042,3204",or,NaN,NaN,105265,111072,134460,107596,134495,154112,150784,92612
carb_entner_1.1.1.47_GLUCOSE__1__DEHYDROGENASE__NAD+__RXN,GDH,NaN,"3003,3042,3204",or,NaN,NaN,105265,111072,134460,107596,134495,154112,150784,92612
carb_entner_1.1.1.360_GLUCOSE__1__DEHYDROGENASE__NADP+__RXN,GDH,NaN,"3003,3042,3204",or,NaN,NaN,105265,111072,134460,107596,134495,154112,150784,92612
carb_entner_3.1.1.17_GLUCONOLACT__RXN,GL,NaN,spontaneous,NaN,NaN,NaN,1000000,1000000,1000000,1000000,1000000,1000000,1000000,1000000


### Calculate the average

In [25]:
# df_match_data_to_reaction_id.to_csv("../data/matched_transcriptone_data_to_reaction_id.csv")

In [7]:
df_match_data_to_reaction_id =pd.read_csv('../data/matched_transcriptone_data_to_reaction_id.csv')
list_columns_70 = [i for i in df_match_data_to_reaction_id.columns if " 70" in i]
list_columns_80 = [i for i in df_match_data_to_reaction_id.columns if " 80" in i]
df_match_data_to_reaction_id["average_list_column_70"] = df_match_data_to_reaction_id[list_columns_70].mean(axis= 1) # calculate the average values of different columns
df_match_data_to_reaction_id["average_list_column_80"] = df_match_data_to_reaction_id[list_columns_80].mean(axis= 1)
df_match_data_to_reaction_id.head()

In [27]:
df_match_data_to_reaction_id.to_csv("../data/reaction_id_RAS.csv")

In [ ]:
s

In [ ]:
model1= model.copy()
df = pd.DataFrame()
df["soxM"] = ""
df["soxABCD"] = ""
df["doxABCDE"] = ""
df["soxM percent"] = ""
df["soxABCD percent"] = ""
df["doxABCDE percent"] = ""
for a in [1,3,5,7,9,11,13,15,100]:
    for rxn in df_gene_reaction_rule.index: 
        if "." in str(df_gene_reaction_rule['Array_67_70°C'].loc[rxn]):

            the_bound = 0.001*df_gene_reaction_rule['Array_67_70°C'].loc[rxn]/a

            if model.reactions.get_by_id(rxn).lower_bound < 0 and model1.reactions.get_by_id(rxn).upper_bound > 0: # this finds reversible rxns

                model1.reactions.get_by_id(rxn).bounds = (-1*the_bound, the_bound)# we set the bound(s) with max abs(FVA) 

            elif model.reactions.get_by_id(rxn).upper_bound > 0: # irreversible, flows in forward direction

                model1.reactions.get_by_id(rxn).bounds = (0, the_bound)  # the lower bound will stay zero

            elif model.reactions.get_by_id(rxn).lower_bound < 0: # irreversible, flows in backward direction

                model1.reactions.get_by_id(rxn).bounds = (-1*the_bound, 0) # the upper bound will stay zero
        else:
            if model.reactions.get_by_id(rxn).lower_bound < 0 and model1.reactions.get_by_id(rxn).upper_bound > 0: # this finds reversible rxns

                model1.reactions.get_by_id(rxn).bounds = (-1000, 1000)# we set the bound(s) with max abs(FVA) 

            if model.reactions.get_by_id(rxn).upper_bound > 0: # irreversible, flows in forward direction

                model1.reactions.get_by_id(rxn).bounds = (0, 1000)  # the lower bound will stay zero

            if model.reactions.get_by_id(rxn).lower_bound < 0: # irreversible, flows in backward direction

                model1.reactions.get_by_id(rxn).bounds = (-1000, 0) 
    model1.objective  = model1.reactions.NGAM
    sol = model1.optimize()
    df.loc[a] = [sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240'],sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240'],sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240'],sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240']/(sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240'] + sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240'] + sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']),sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240']/(sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']),sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']/(sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240'])]    

## HU treatment  transcript

### excel that contains the HU treatment  transcript
- The data is from the paper 'HU treatment of Sulfolobus solfataricus P2'.
- Samples were taken from the cells growing with or without the treatment of Hydroxyurea (HU) for 4 hours.
- The data can be download from the website (https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE87612)
- As this transcript data doesn't contain the information of SSO numbers, and it contains the gene location information so we used a file with SSO numbers and also gene location information to map the SSO numbers to the transcript data.

In [ ]:
# add tge gene reaction rule to the model
# reactions_ids = [rxn.id for rxn in model.reactions]
df_genome=pd.read_table('../data/GCA_000007005.1_ASM700v1_feature_table.txt') # This file contains the gene information of sulfolubus. 
df_HU_treatment =pd.read_excel('../data/transciptone.xlsx',sheet_name = "HU treatment of sulfolobus ")
df_genome = df_genome.set_index(["# feature"],)
df_genome = df_genome.drop(index='gene')
df_HU_treatment = df_HU_treatment.set_index(["START"],)
df_HU_treatment["ORF"] = ""
for start in df_HU_treatment.index:
    for i in range(len(df_genome)):
        if start == df_genome["start"].iloc[i]: 
            df_HU_treatment["ORF"].at[start] = (df_genome["locus_tag"].iloc[i]).strip()     
df_HU_treatment = df_HU_treatment.set_index("ORF")
df_gene_reaction_rule["HU TREATED/CONTROL"] = ""
df_gene_reaction_rule["RAS_HU TREATED/CONTROL"] = ""

df_gene_reaction_rule["gene_reaction_relation"] = ""
for rxn in df_gene_reaction_rule.index:
    
    if rxn != "oxp_redox_1.6.5.3_RXN0__5330":
        if 'spontaneous' not in str(df_gene_reaction_rule['genes'].loc[rxn]):
            c = df_gene_reaction_rule["genes"].loc[rxn].split(",")
            if len(c) > 1:
                gene_rules = []
                gene_numbers1 = []
                gene_numbers2 = []
                
                for i in range(len(c)):
                    e = "SSO" + c[i]
                    if e in df_HU_treatment.index:
#                         print (e)
                        gene_rules.append(e)
#                         ge = df_HU_treatment["HU TREATED"].loc[e]/df_HU_treatment["CONTROL"].loc[e]
                       
                        gene_numbers1.append(df_HU_treatment["HU TREATED/CONTROL"].loc[e])
#                         gene_numbers2.append(df_HU_treatment["CONTROL"].loc[e])
                  
                if "or" in df_gene_reaction_rule['gene_reaction_rule'].loc[rxn]:
                    print (gene_numbers1)
                    df_gene_reaction_rule["RAS_HU TREATED/CONTROL"].loc[rxn] = sum(gene_numbers1)

#                     df_gene_reaction_rule["RAS_control"].loc[rxn] = sum(gene_numbers2)
#                 if "and" in str(df_gene_reaction_rule['gene_reaction_rule'].loc[rxn]):
#                     df_gene_reaction_rule["RAS_HU TREATED/CONTROL"].loc[rxn] = min(gene_numbers1)

#                     df_gene_reaction_rule["RAS_control"].loc[rxn] = min(gene_numbers2)
                    


In [ ]:
# add tge gene reaction rule to the model
# reactions_ids = [rxn.id for rxn in model.reactions]
df_genome=pd.read_table('../data/GCA_000007005.1_ASM700v1_feature_table.txt') # This file contains the gene information of sulfolubus. 
df_HU_treatment =pd.read_excel('../data/transciptone.xlsx',sheet_name = "HU treatment of sulfolobus ")
df_genome = df_genome.set_index(["# feature"],)
df_genome = df_genome.drop(index='gene')
df_HU_treatment = df_HU_treatment.set_index(["START"],)
df_HU_treatment["ORF"] = ""
for start in df_HU_treatment.index:
    for i in range(len(df_genome)):
        if start == df_genome["start"].iloc[i]: 
            df_HU_treatment["ORF"].at[start] = (df_genome["locus_tag"].iloc[i]).strip()     
df_HU_treatment = df_HU_treatment.set_index("ORF")
df_gene_reaction_rule["HU TREATED/CONTROL"] = ""
df_gene_reaction_rule["RAS_HU TREATED/CONTROL"] = ""

df_gene_reaction_rule["gene_reaction_relation"] = ""
for rxn in df_gene_reaction_rule.index:
    
    if rxn != "oxp_redox_1.6.5.3_RXN0__5330":
        if 'spontaneous' not in str(df_gene_reaction_rule['genes'].loc[rxn]):
            c = df_gene_reaction_rule["genes"].loc[rxn].split(",")
            if len(c) > 1:
                gene_rules = []
                gene_numbers1 = []
                gene_numbers2 = []
                
                for i in range(len(c)):
                    e = "SSO" + c[i]
                    if e in df_HU_treatment.index:
#                         print (e)
                        gene_rules.append(e)
#                         ge = df_HU_treatment["HU TREATED"].loc[e]/df_HU_treatment["CONTROL"].loc[e]
                       
                        gene_numbers1.append(df_HU_treatment["HU TREATED/CONTROL"].loc[e])
#                         gene_numbers2.append(df_HU_treatment["CONTROL"].loc[e])
                  
                if "or" in df_gene_reaction_rule['gene_reaction_rule'].loc[rxn]:
                    df_gene_reaction_rule["RAS_HU TREATED/CONTROL"].loc[rxn] = sum(gene_numbers1)

#                     df_gene_reaction_rule["RAS_control"].loc[rxn] = sum(gene_numbers2)
                if "and" in str(df_gene_reaction_rule['gene_reaction_rule'].loc[rxn]):
                    df_gene_reaction_rule["RAS_HU TREATED/CONTROL"].loc[rxn] = min(gene_numbers1)

#                     df_gene_reaction_rule["RAS_control"].loc[rxn] = min(gene_numbers2)
                    

        
                d = str(" "+ str(df_gene_reaction_rule['gene_reaction_rule'].loc[rxn]) + " ").join(gene_rules)
                df_gene_reaction_rule["HU TREATED/CONTROL"].loc[rxn] = gene_numbers1
#                 df_gene_reaction_rule["control"].loc[rxn] = gene_numbers2
                df_gene_reaction_rule["gene_reaction_relation"].loc[rxn] = "(" + d + ')'
            elif len(c) == 1:
                gene_rules = []
                gene_numbers1 = []
                gene_numbers2 = []
                
                e = "SSO" + c[0]
                if e in df_HU_treatment.index:
                    d = e 
                    gene_numbers1.append(df_HU_treatment["HU TREATED/CONTROL"].loc[e])
#                     gene_numbers2.append(df_HU_treatment["CONTROL"].loc[e])
                   
                    df_gene_reaction_rule["HU TREATED/CONTROL"].loc[rxn] = gene_numbers1
#                     df_gene_reaction_rule["control"].loc[rxn] = gene_numbers2
                    df_gene_reaction_rule["gene_reaction_relation"].loc[rxn] = d
                    df_gene_reaction_rule["RAS_HU TREATED/CONTROL"].loc[rxn] = df_HU_treatment["HU TREATED/CONTROL"].loc[e]
#                     df_gene_reaction_rule["RAS_control"].loc[rxn] = df_HU_treatment["CONTROL"].loc[e]
        else:
            
            df_gene_reaction_rule["HU TREATED/CONTROL"].loc[rxn] = 1000
#             df_gene_reaction_rule["RAS_control"].loc[rxn] = 1000
                    
           
    elif rxn == "oxp_redox_1.6.5.3_RXN0__5330":
        g = df_gene_reaction_rule["genes"].loc[rxn].split("or")
        gene_rules = []
        gene_numbers1 = []
        gene_numbers2 = []
        c1 = g[0].split(",")
        for i in range(len(c1)):
        
            e = "SSO" + c1[i]
            if e in df_HU_treatment.index:
#  
                gene_rules.append(e)
                gene_numbers1.append(df_HU_treatment["HU TREATED/CONTROL"].loc[e])
#                 gene_numbers2.append(df_HU_treatment["CONTROL"].loc[e])
        
               
        
               
            
        d1 = str(" "+ str(df_gene_reaction_rule['gene_reaction_rule'].loc[rxn]) + " ").join(gene_rules)
        gene_numbers3 = []
        gene_numbers4 = []
        c2 = g[1].split(",")
        for i in range(len(c2)):
        
            e = "SSO" + c2[i]
            if e in df_HU_treatment.index:  
                gene_numbers3.append(df_HU_treatment["HU TREATED/CONTROL"].loc[e])
#                 gene_numbers4.append(df_HU_treatment["CONTROL"].loc[e])
        
               
        df_gene_reaction_rule["HU TREATED/CONTROL"].loc[rxn] = gene_numbers1 + gene_numbers3
#         df_gene_reaction_rule["control"].loc[rxn] = gene_numbers2 + gene_numbers4
        d2 = str(" "+ str(df_gene_reaction_rule['gene_reaction_rule'].loc[rxn]) + " ").join(gene_rules)
        
        df_gene_reaction_rule["gene_reaction_relation"].loc[rxn] = "((" + d1 + ") or (" +  d2+ "))" 
        df_gene_reaction_rule["RAS_HU TREATED/CONTROL"].loc[rxn] = min(gene_numbers1) + min(gene_numbers3)
#         df_gene_reaction_rule["RAS_control"].loc[rxn] = min(gene_numbers2) + min(gene_numbers4)

In [ ]:
df_gene_reaction_rule

In [ ]:
df_gene_reaction_rule ["RAS_HU TREATED/CONTROL"]

### RAS_HU TREATED/CONTROL

In [ ]:
model1 = M.copy()
df = pd.DataFrame()
df["soxM"] = ""
df["soxABCD"] = ""
df["doxABCDE"] = ""
df["soxM percent"] = ""
df["soxABCD percent"] = ""
df["doxABCDE percent"] = ""
df["soxM bounds"] = ""
df["soxABCD bounds"] = ""
df["doxABCDE bounds"] = ""
df["ATPase bounds"] = ""
df["NGAM"] = ""
df["ENO"] = ""
df["ATPase"] = ""
for a in [1,3,5,7,9,11,13,15]:
    for rxn in df_gene_reaction_rule.index: 
        if "." in str(df_gene_reaction_rule['RAS_HU TREATED/CONTROL'].loc[rxn]):

            the_bound = a*df_gene_reaction_rule['RAS_HU TREATED/CONTROL'].loc[rxn]

            if model.reactions.get_by_id(rxn).lower_bound < 0 and model.reactions.get_by_id(rxn).upper_bound > 0: # this finds reversible rxns

                model1.reactions.get_by_id(rxn).bounds = (-1*the_bound, the_bound)# we set the bound(s) with max abs(FVA) 

            elif model.reactions.get_by_id(rxn).upper_bound > 0: # irreversible, flows in forward direction

                model1.reactions.get_by_id(rxn).bounds = (0, the_bound)  # the lower bound will stay zero

            elif model.reactions.get_by_id(rxn).lower_bound < 0: # irreversible, flows in backward direction

                model1.reactions.get_by_id(rxn).bounds = (-1*the_bound, 0) # the upper bound will stay zero
        else:
            if model.reactions.get_by_id(rxn).lower_bound < 0 and model.reactions.get_by_id(rxn).upper_bound > 0: # this finds reversible rxns

                model1.reactions.get_by_id(rxn).bounds = (-1000, 1000)# we set the bound(s) with max abs(FVA) 

            if model.reactions.get_by_id(rxn).upper_bound > 0: # irreversible, flows in forward direction

                model1.reactions.get_by_id(rxn).bounds = (0, 1000)  # the lower bound will stay zero

            if model.reactions.get_by_id(rxn).lower_bound < 0: # irreversible, flows in backward direction

                model1.reactions.get_by_id(rxn).bounds = (-1000, 0) 
    model1.objective  = model1.reactions.NGAM
    sol = model1.optimize()
#     df.loc[a] = [sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240'],sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240'],sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240'],sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240']/(sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240'] + sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240'] + sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']),sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240']/(sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']),sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']/(sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']),model1.reactions.get_by_id('oxp_redox_1.10.3.13_soxM_TRANS__RXN__240').bounds,model1.reactions.get_by_id('oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240').bounds,model1.reactions.get_by_id('oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240').bounds,sol.fluxes.loc['NGAM'],sol.fluxes.loc['carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN'],sol.fluxes.loc['oxp_3.6.3.14_ATP__synthase']]        
    df.loc[a] = [sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240'],sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240'],sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240'],sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240']/(sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240'] + sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240'] + sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']),sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240']/(sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']),sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']/(sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']),model1.reactions.get_by_id('oxp_redox_1.10.3.13_soxM_TRANS__RXN__240').bounds,model1.reactions.get_by_id('oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240').bounds,model1.reactions.get_by_id('oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240').bounds,model1.reactions.get_by_id('oxp_3.6.3.14_ATP__synthase').bounds,sol.fluxes.loc['NGAM'],sol.fluxes.loc['carb_enter_glucneo_4.2.1.11_2PGADEHYDRAT__RXN'],sol.fluxes.loc['oxp_3.6.3.14_ATP__synthase']]    

In [ ]:
df

In [ ]:
for rxn in df_gene_reaction_rule.index: 
    if "." in str(df_gene_reaction_rule['RAS_HU TREATED/CONTROL'].loc[rxn]):

        the_bound = 9*df_gene_reaction_rule['RAS_HU TREATED/CONTROL'].loc[rxn]

        if model.reactions.get_by_id(rxn).lower_bound < 0 and model1.reactions.get_by_id(rxn).upper_bound > 0: # this finds reversible rxns

            model1.reactions.get_by_id(rxn).bounds = (-1*the_bound, the_bound)# we set the bound(s) with max abs(FVA) 

        elif model.reactions.get_by_id(rxn).upper_bound > 0: # irreversible, flows in forward direction

            model1.reactions.get_by_id(rxn).bounds = (0, the_bound)  # the lower bound will stay zero

        elif model.reactions.get_by_id(rxn).lower_bound < 0: # irreversible, flows in backward direction

            model1.reactions.get_by_id(rxn).bounds = (-1*the_bound, 0) # the upper bound will stay zero
    else:
        if model.reactions.get_by_id(rxn).lower_bound < 0 and model1.reactions.get_by_id(rxn).upper_bound > 0: # this finds reversible rxns

            model1.reactions.get_by_id(rxn).bounds = (-1000, 1000)# we set the bound(s) with max abs(FVA) 

        if model.reactions.get_by_id(rxn).upper_bound > 0: # irreversible, flows in forward direction

            model1.reactions.get_by_id(rxn).bounds = (0, 1000)  # the lower bound will stay zero

        if model.reactions.get_by_id(rxn).lower_bound < 0: # irreversible, flows in backward direction

            model1.reactions.get_by_id(rxn).bounds = (-1000, 0) 
model1.objective  = model1.reactions.NGAM
sol = model1.optimize()
l = sol.fluxes[abs(sol.fluxes)>1e-6].index

for rxn in l:
    print (rxn,model1.reactions.get_by_id(rxn).reaction,model1.reactions.get_by_id(rxn).bounds,sol.fluxes.loc[rxn])
b = show_map(sol,map_loc)
b

In [ ]:
model1.reactions.T_flux_ammonium.bounds

## Temperature_treatment transcript 
- Import the excel that contain the transcript data with shifting temperature from 80 ℃ to 90 ℃.
- The data is from the paper 'Dynamic Metabolic Adjustments and Genome Plasticity Are Implicated in the Heat Shock Response of the Extremely Thermoacidophilic Archaeon Sulfolobus solfataricus'.
- Samples were taken 10 min (-10) before starting the temperature shift and then 5, 30, and 60 min after reaching 90°C.
- The data can be download from the website (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1482968/bin/jbacter_188_12_4553__index.html)

In [ ]:
df_gene_reaction_rule["Array_67_70°C"] = ""
df_gene_reaction_rule["Array_67_80°C"] = ""

df_gene_reaction_rule["gene_reaction_relation"] = ""
for rxn in df_gene_reaction_rule.index:
    
    if rxn != "oxp_redox_1.6.5.3_RXN0__5330":
        if 'spontaneous' not in str(df_gene_reaction_rule['genes'].loc[rxn]):
            c = df_gene_reaction_rule["genes"].loc[rxn].split(",")
            if len(c) > 1:
                gene_rules = []
                gene_numbers1 = []
                gene_numbers2 = []
                
                for i in range(len(c)):
                    e = "SSO" + c[i]
                    if e in df_temperature_treatment.index:
                        print (e)
                        gene_numbers1.append(df_temperature_treatment["Array_67_70°C"].loc[e])
                        gene_numbers2.append(df_temperature_treatment["Array_67_80°C"].loc[e])
                  
                if "or" in df_gene_reaction_rule['gene_reaction_rule'].loc[rxn]:
                    df_gene_reaction_rule["Array_67_70°C"].loc[rxn] = sum(gene_numbers1)
                    df_gene_reaction_rule["Array_67_80°C"].loc[rxn] = sum(gene_numbers2)
                if "and" in str(df_gene_reaction_rule['gene_reaction_rule'].loc[rxn]):
                    df_gene_reaction_rule["Array_67_70°C"].loc[rxn] = min(gene_numbers1)
                    df_gene_reaction_rule["Array_67_80°C"].loc[rxn] = min(gene_numbers2)
                    

            elif len(c) == 1:
                gene_rules = []
                gene_numbers1 = []
                gene_numbers2 = []
                
                e = "SSO" + c[0]
                if e in df_temperature_treatment.index:
                    d = e 
                    gene_numbers1.append(df_temperature_treatment["Array_67_70°C"].loc[e])
                    gene_numbers2.append(df_temperature_treatment["Array_67_80°C"].loc[e])
                    df_gene_reaction_rule["Array_67_70°C"].loc[rxn] = df_temperature_treatment["Array_67_80°C"].loc[e]
                    df_gene_reaction_rule["Array_67_80°C"].loc[rxn] = df_temperature_treatment["Array_67_80°C"].loc[e]
        else:
            
            df_gene_reaction_rule["Array_67_70°C"].loc[rxn] = 1000
            df_gene_reaction_rule["Array_67_80°C"].loc[rxn] = 1000
                             
    elif rxn == "oxp_redox_1.6.5.3_RXN0__5330":
        g = df_gene_reaction_rule["genes"].loc[rxn].split("or")
        gene_rules = []
        gene_numbers1 = []
        gene_numbers2 = []
        c1 = g[0].split(",")
        for i in range(len(c1)):
        
            e = "SSO" + c1[i]
            if e in df_temperature_treatment.index:
                gene_rules.append(e)
                gene_numbers1.append(df_temperature_treatment["Array_67_70°C"].loc[e])
                gene_numbers2.append(df_temperature_treatment["Array_67_80°C"].loc[e])               
            
        d1 = str(" "+ str(df_gene_reaction_rule['gene_reaction_rule'].loc[rxn]) + " ").join(gene_rules)
        gene_numbers3 = []
        gene_numbers4 = []
        c2 = g[1].split(",")
        for i in range(len(c2)):
        
            e = "SSO" + c2[i]
            if e in df_temperature_treatment.index:
                gene_rules.append(e)
                gene_numbers3.append(df_temperature_treatment["Array_67_70°C"].loc[e])
                gene_numbers4.append(df_temperature_treatment["Array_67_80°C"].loc[e])

        df_gene_reaction_rule["Array_67_70°C"].loc[rxn] = min(gene_numbers1) + min(gene_numbers3)

        df_gene_reaction_rule["Array_67_80°C"].loc[rxn] = min(gene_numbers2) + min(gene_numbers4)
    

### excel that contains the temperature treatment  transcript

In [ ]:
df_dynamic_shift =pd.read_excel('../data/transciptone.xlsx',sheet_name = "dynamic shift")
df_dynamic_shift = df_dynamic_shift.set_index (["ORF"])
# df_gene_reaction_rule["SSO_5 vs. -10"] = ""
df_gene_reaction_rule["RAS_5 vs. -10"] = ""
# df_gene_reaction_rule["SSO_30 vs. 5"] = ""
df_gene_reaction_rule["RAS_30 vs. 5"] = ""
# df_gene_reaction_rule["SSO_60 vs. 30"] = ""
df_gene_reaction_rule["RAS_60 vs. 30"] = ""
# df_gene_reaction_rule["SSO_60 vs.  -10"] = ""
df_gene_reaction_rule["RAS_60 vs.  -10"] = ""
df_gene_reaction_rule["gene_reaction_relation"] = ""
for rxn in df_gene_reaction_rule.index:
    
    if rxn != "oxp_redox_1.6.5.3_RXN0__5330":
        if 'spontaneous' not in str(df_gene_reaction_rule['genes'].loc[rxn]):
            c = df_gene_reaction_rule["genes"].loc[rxn].split(",")
            if len(c) > 1:
                gene_rules = []
                gene_numbers1 = []
                gene_numbers2 = []
                gene_numbers3 = []
                gene_numbers4 = []
                for i in range(len(c)):
                    e = "SSO" + c[i]
                    if e in df_dynamic_shift.index: 
                        gene_rules.append(e)
                       
                        gene_numbers1.append(df_dynamic_shift["5 vs.  -10"].loc[e])
                        gene_numbers2.append(df_dynamic_shift["30 vs. 5 "].loc[e])
                        gene_numbers3.append(df_dynamic_shift["60 vs. 30"].loc[e])
                        gene_numbers4.append(df_dynamic_shift["60 vs.  -10"].loc[e])
                if "or" in df_gene_reaction_rule['gene_reaction_rule'].loc[rxn]:
                    df_gene_reaction_rule["RAS_5 vs. -10"].loc[rxn] = sum(gene_numbers1)
                    df_gene_reaction_rule["RAS_30 vs. 5"].loc[rxn] = sum(gene_numbers2)
                    df_gene_reaction_rule["RAS_60 vs. 30"].loc[rxn] = sum(gene_numbers3)
                    df_gene_reaction_rule["RAS_60 vs.  -10"].loc[rxn] = sum(gene_numbers4)

                if "and" in str(df_gene_reaction_rule['gene_reaction_rule'].loc[rxn]):
                    df_gene_reaction_rule["RAS_5 vs. -10"].loc[rxn] = min(gene_numbers1)
                    df_gene_reaction_rule["RAS_30 vs. 5"].loc[rxn] = min(gene_numbers2)
                    df_gene_reaction_rule["RAS_60 vs. 30"].loc[rxn] = min(gene_numbers3)
                    df_gene_reaction_rule["RAS_60 vs.  -10"].loc[rxn] = min(gene_numbers4)
                d = str(" "+ str(df_gene_reaction_rule['gene_reaction_rule'].loc[rxn]) + " ").join(gene_rules)
                
                df_gene_reaction_rule["gene_reaction_relation"].loc[rxn] = "(" + d + ')'
            elif len(c) == 1:   
                e = "SSO" + c[0]
                if e in df_dynamic_shift.index:
                    df_gene_reaction_rule["gene_reaction_relation"].loc[rxn] = e
                    df_gene_reaction_rule["RAS_5 vs. -10"].loc[rxn] = df_dynamic_shift["5 vs.  -10"].loc[e]
                    df_gene_reaction_rule["RAS_30 vs. 5"].loc[rxn] = df_dynamic_shift["30 vs. 5 "].loc[e]
                    df_gene_reaction_rule["RAS_60 vs. 30"].loc[rxn] = df_dynamic_shift["60 vs. 30"].loc[e]
                    df_gene_reaction_rule["RAS_60 vs.  -10"].loc[rxn] = df_dynamic_shift["60 vs.  -10"].loc[e]
                    
        else:
            
            df_gene_reaction_rule["RAS_5 vs. -10"].loc[rxn] = 1000    
            df_gene_reaction_rule["RAS_30 vs. 5"].loc[rxn] = 1000 
            df_gene_reaction_rule["RAS_60 vs. 30"].loc[rxn] = 1000 
            df_gene_reaction_rule["RAS_60 vs.  -10"].loc[rxn] = 1000   
    elif rxn == "oxp_redox_1.6.5.3_RXN0__5330":
        g = df_gene_reaction_rule["genes"].loc[rxn].split("or")
        gene_rules = []
        gene_numbers1 = []
        gene_numbers2 = []
        gene_numbers3 = []
        gene_numbers4 = []
        c1 = g[0].split(",")
        for i in range(len(c1)):
        
            e = "SSO" + c1[i]
            if e in df_dynamic_shift.index:  
                gene_rules.append(e)
                gene_numbers1.append(df_dynamic_shift["5 vs.  -10"].loc[e])
                gene_numbers2.append(df_dynamic_shift["30 vs. 5 "].loc[e])
                gene_numbers3.append(df_dynamic_shift["60 vs. 30"].loc[e])
                gene_numbers4.append(df_dynamic_shift["60 vs.  -10"].loc[e])
                
        d1 = str(" "+ str(df_gene_reaction_rule['gene_reaction_rule'].loc[rxn]) + " ").join(gene_rules)
        
        c2 = g[1].split(",")
        for i in range(len(c2)):
            gene_numbers5 = []
            gene_numbers6 = []
            gene_numbers7 = []
            gene_numbers8 = []
            e = "SSO" + c2[i]
            if e in df_dynamic_shift.index:  
                gene_numbers5.append(df_dynamic_shift["5 vs.  -10"].loc[e])
                gene_numbers6.append(df_dynamic_shift["30 vs. 5 "].loc[e])
                gene_numbers7.append(df_dynamic_shift["60 vs. 30"].loc[e])
                gene_numbers8.append(df_dynamic_shift["60 vs.  -10"].loc[e])

               
        
        d2 = str(" "+ str(df_gene_reaction_rule['gene_reaction_rule'].loc[rxn]) + " ").join(gene_rules)
        
        df_gene_reaction_rule["gene_reaction_relation"].loc[rxn] = "((" + d1 + ") or (" +  d2+ "))" 
        df_gene_reaction_rule["RAS_5 vs. -10"].loc[rxn] = min(gene_numbers1) + min(gene_numbers5)
        df_gene_reaction_rule["RAS_30 vs. 5"].loc[rxn] = min(gene_numbers2) + min(gene_numbers6)
        df_gene_reaction_rule["RAS_60 vs. 30"].loc[rxn] = min(gene_numbers3) + min(gene_numbers7)
        df_gene_reaction_rule["RAS_60 vs.  -10"].loc[rxn] = min(gene_numbers4) + min(gene_numbers8)
        

### RAS_5 vs.  -10

In [ ]:
model1= M.copy()
df = pd.DataFrame()
df["soxM"] = ""
df["soxABCD"] = ""
df["doxABCDE"] = ""
df["soxM percent"] = ""
df["soxABCD percent"] = ""
df["doxABCDE percent"] = ""
for a in [1,3,5,7,9,11,13,15,100]:
    for rxn in df_gene_reaction_rule.index: 
        if "." in str(df_gene_reaction_rule['RAS_5 vs. -10'].loc[rxn]):

            the_bound = a*df_gene_reaction_rule['RAS_5 vs. -10'].loc[rxn]

            if model.reactions.get_by_id(rxn).lower_bound < 0 and model1.reactions.get_by_id(rxn).upper_bound > 0: # this finds reversible rxns

                model1.reactions.get_by_id(rxn).bounds = (-1*the_bound, the_bound)# we set the bound(s) with max abs(FVA) 

            elif model.reactions.get_by_id(rxn).upper_bound > 0: # irreversible, flows in forward direction

                model1.reactions.get_by_id(rxn).bounds = (0, the_bound)  # the lower bound will stay zero

            elif model.reactions.get_by_id(rxn).lower_bound < 0: # irreversible, flows in backward direction

                model1.reactions.get_by_id(rxn).bounds = (-1*the_bound, 0) # the upper bound will stay zero
        else:
            if model.reactions.get_by_id(rxn).lower_bound < 0 and model1.reactions.get_by_id(rxn).upper_bound > 0: # this finds reversible rxns

                model1.reactions.get_by_id(rxn).bounds = (-1000, 1000)# we set the bound(s) with max abs(FVA) 

            if model.reactions.get_by_id(rxn).upper_bound > 0: # irreversible, flows in forward direction

                model1.reactions.get_by_id(rxn).bounds = (0, 1000)  # the lower bound will stay zero

            if model.reactions.get_by_id(rxn).lower_bound < 0: # irreversible, flows in backward direction

                model1.reactions.get_by_id(rxn).bounds = (-1000, 0) 
    model1.objective  = model1.reactions.NGAM
    sol = model1.optimize()
    df.loc[a] = [sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240'],sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240'],sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240'],sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240']/(sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240'] + sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240'] + sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']),sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240']/(sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']),sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']/(sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240'])]    

In [ ]:
df

### RAS_30 vs. 5

In [ ]:
model1= M.copy()
df = pd.DataFrame()
df["soxM"] = ""
df["soxABCD"] = ""
df["doxABCDE"] = ""
df["soxM percent"] = ""
df["soxABCD percent"] = ""
df["doxABCDE percent"] = ""
for a in [1,3,5,7,9,11,13,15,100]:
    for rxn in df_gene_reaction_rule.index: 
        if "." in str(df_gene_reaction_rule['RAS_30 vs. 5'].loc[rxn]):

            the_bound = a*df_gene_reaction_rule['RAS_30 vs. 5'].loc[rxn]

            if model.reactions.get_by_id(rxn).lower_bound < 0 and model1.reactions.get_by_id(rxn).upper_bound > 0: # this finds reversible rxns

                model1.reactions.get_by_id(rxn).bounds = (-1*the_bound, the_bound)# we set the bound(s) with max abs(FVA) 

            elif model.reactions.get_by_id(rxn).upper_bound > 0: # irreversible, flows in forward direction

                model1.reactions.get_by_id(rxn).bounds = (0, the_bound)  # the lower bound will stay zero

            elif model.reactions.get_by_id(rxn).lower_bound < 0: # irreversible, flows in backward direction

                model1.reactions.get_by_id(rxn).bounds = (-1*the_bound, 0) # the upper bound will stay zero
        else:
            if model.reactions.get_by_id(rxn).lower_bound < 0 and model1.reactions.get_by_id(rxn).upper_bound > 0: # this finds reversible rxns

                model1.reactions.get_by_id(rxn).bounds = (-1000, 1000)# we set the bound(s) with max abs(FVA) 

            if model.reactions.get_by_id(rxn).upper_bound > 0: # irreversible, flows in forward direction

                model1.reactions.get_by_id(rxn).bounds = (0, 1000)  # the lower bound will stay zero

            if model.reactions.get_by_id(rxn).lower_bound < 0: # irreversible, flows in backward direction

                model1.reactions.get_by_id(rxn).bounds = (-1000, 0) 
    model1.objective  = model1.reactions.NGAM
    sol = model1.optimize()
    df.loc[a] = [sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240'],sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240'],sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240'],sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240']/(sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240'] + sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240'] + sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']),sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240']/(sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']),sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']/(sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240'])]    

In [ ]:
df

### RAS_60 vs. 30

In [ ]:
model1= M.copy()
df = pd.DataFrame()
df["soxM"] = ""
df["soxABCD"] = ""
df["doxABCDE"] = ""
df["soxM percent"] = ""
df["soxABCD percent"] = ""
df["doxABCDE percent"] = ""
for a in [1,3,5,7,9,11,13,15,100]:
    for rxn in df_gene_reaction_rule.index: 
        if "." in str(df_gene_reaction_rule['RAS_60 vs. 30'].loc[rxn]):

            the_bound = a*df_gene_reaction_rule['RAS_60 vs. 30'].loc[rxn]

            if model.reactions.get_by_id(rxn).lower_bound < 0 and model1.reactions.get_by_id(rxn).upper_bound > 0: # this finds reversible rxns

                model1.reactions.get_by_id(rxn).bounds = (-1*the_bound, the_bound)# we set the bound(s) with max abs(FVA) 

            elif model.reactions.get_by_id(rxn).upper_bound > 0: # irreversible, flows in forward direction

                model1.reactions.get_by_id(rxn).bounds = (0, the_bound)  # the lower bound will stay zero

            elif model.reactions.get_by_id(rxn).lower_bound < 0: # irreversible, flows in backward direction

                model1.reactions.get_by_id(rxn).bounds = (-1*the_bound, 0) # the upper bound will stay zero
        else:
            if model.reactions.get_by_id(rxn).lower_bound < 0 and model1.reactions.get_by_id(rxn).upper_bound > 0: # this finds reversible rxns

                model1.reactions.get_by_id(rxn).bounds = (-1000, 1000)# we set the bound(s) with max abs(FVA) 

            if model.reactions.get_by_id(rxn).upper_bound > 0: # irreversible, flows in forward direction

                model1.reactions.get_by_id(rxn).bounds = (0, 1000)  # the lower bound will stay zero

            if model.reactions.get_by_id(rxn).lower_bound < 0: # irreversible, flows in backward direction

                model1.reactions.get_by_id(rxn).bounds = (-1000, 0) 
    model1.objective  = model1.reactions.NGAM
    sol = model1.optimize()
    df.loc[a] = [sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240'],sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240'],sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240'],sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240']/(sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240'] + sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240'] + sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']),sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240']/(sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']),sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']/(sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240'])]    

In [ ]:
df

### RAS_60 vs.  -10

In [ ]:
model1= M.copy()
df = pd.DataFrame()
df["soxM"] = ""
df["soxABCD"] = ""
df["doxABCDE"] = ""
df["soxM percent"] = ""
df["soxABCD percent"] = ""
df["doxABCDE percent"] = ""
for a in [1,3,5,7,9,11,13,15,100]:
    for rxn in df_gene_reaction_rule.index: 
        if "." in str(df_gene_reaction_rule['RAS_60 vs.  -10'].loc[rxn]):

            the_bound = a*df_gene_reaction_rule['RAS_60 vs.  -10'].loc[rxn]

            if model.reactions.get_by_id(rxn).lower_bound < 0 and model1.reactions.get_by_id(rxn).upper_bound > 0: # this finds reversible rxns

                model1.reactions.get_by_id(rxn).bounds = (-1*the_bound, the_bound)# we set the bound(s) with max abs(FVA) 

            elif model.reactions.get_by_id(rxn).upper_bound > 0: # irreversible, flows in forward direction

                model1.reactions.get_by_id(rxn).bounds = (0, the_bound)  # the lower bound will stay zero

            elif model.reactions.get_by_id(rxn).lower_bound < 0: # irreversible, flows in backward direction

                model1.reactions.get_by_id(rxn).bounds = (-1*the_bound, 0) # the upper bound will stay zero
        else:
            if model.reactions.get_by_id(rxn).lower_bound < 0 and model1.reactions.get_by_id(rxn).upper_bound > 0: # this finds reversible rxns

                model1.reactions.get_by_id(rxn).bounds = (-1000, 1000)# we set the bound(s) with max abs(FVA) 

            if model.reactions.get_by_id(rxn).upper_bound > 0: # irreversible, flows in forward direction

                model1.reactions.get_by_id(rxn).bounds = (0, 1000)  # the lower bound will stay zero

            if model.reactions.get_by_id(rxn).lower_bound < 0: # irreversible, flows in backward direction

                model1.reactions.get_by_id(rxn).bounds = (-1000, 0) 
    model1.objective  = model1.reactions.NGAM
    sol = model1.optimize()
    df.loc[a] = [sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240'],sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240'],sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240'],sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240']/(sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240'] + sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240'] + sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']),sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240']/(sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']),sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']/(sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240'])]    

In [ ]:
df

## Different oxygen treatment transcript
- The data is from the paper 'Effect of O2 concentrations on Sulfolobus solfataricus P2'.
- Sulfolobus solfataricus P2 was grown aerobically, with O2 concentrations ranging from 1.5 to 26 % (v/v; gas phase). To gain some insight in control of the respiratory system, transcriptomes of the strain cultivated in different O2 concentrations (1.5 % vs 21 %, 1.5 % vs 26 %) were compared using a DNA microarray approach.
- The data can be download from the website (https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE16043)

### excel that contains the oxygen treatment

In [ ]:
#Import the excel file that contain the oxygen treatment transcript data.
df_oxygen_transcript =pd.read_excel('../data/transciptone.xlsx',sheet_name = "Oxygen treatment")
df_oxygen_transcript = df_oxygen_transcript.loc[df_oxygen_transcript['gene'].str.contains('SSO') == True] # Drop out the rows without SSO numbers.
df_oxygen_transcript["ORF"] = ""
df_oxygen_transcript["position"] = ""
for i in range(len(df_oxygen_transcript)):
    df_oxygen_transcript['ORF'].iat[i] = df_oxygen_transcript["gene"].iloc[i].split("_")[0]
    df_oxygen_transcript["position"].iat[i] = df_oxygen_transcript["gene"].iloc[i].split("_")[1].split(" ")[0]
df_oxygen_transcript = df_oxygen_transcript.set_index(["ORF"],)
df_oxygen_transcript["1.5vs21"] = df_oxygen_transcript[["1.5vs21 hyb_1","1.5vs21 hyb_2","1.5vs21 hyb_3","21vs1.5 hyb_1",'21vs1.5 hyb_2',"21vs1.5 hyb_3"]].mean(axis = 1)
df_oxygen_transcript["1.5vs26"] = df_oxygen_transcript[["1.5vs26 hyb_1","1.5vs26 hyb_2","1.5vs26 hyb_3","26vs1.5 hyb_1",'26vs1.5 hyb_2',"26vs1.5 hyb_3"]].mean(axis = 1)

df_gene_reaction_rule["Oxygen_treatment_1.5vs26 hyb transcript"] = ""
df_gene_reaction_rule["Oxygen_treatment_1.5vs21 hyb transcript"] = ""
df_gene_reaction_rule["RAS_Oxygen_treatment_1.5vs26 hyb transcript"] = ""
df_gene_reaction_rule["RAS_Oxygen_treatment_1.5vs21 hyb transcript"] = ""
for rxn in df_gene_reaction_rule.index:  
    if rxn != "oxp_redox_1.6.5.3_RXN0__5330":
        if 'spontaneous' not in str(df_gene_reaction_rule['genes'].loc[rxn]):
            c = df_gene_reaction_rule["genes"].loc[rxn].split(",")
            if len(c) > 1:
                gene_numbers1 = []
                gene_numbers2 = []
                
                for i in range(len(c)):
                    e = "SSO" + c[i]
                    if e in df_oxygen_transcript.index:
#                        
                        gene_numbers1.append(df_oxygen_transcript["1.5vs26"].loc[e])
                        gene_numbers2.append(df_oxygen_transcript["1.5vs21"].loc[e])    
                        
                df_gene_reaction_rule["Oxygen_treatment_1.5vs26 hyb transcript"].loc[rxn] = gene_numbers1
                df_gene_reaction_rule["Oxygen_treatment_1.5vs21 hyb transcript"].loc[rxn] = gene_numbers2
                
                if "or" in df_gene_reaction_rule['gene_reaction_rule'].loc[rxn]:
                    df_gene_reaction_rule["RAS_Oxygen_treatment_1.5vs26 hyb transcript"].loc[rxn] = sum(gene_numbers1)
                    df_gene_reaction_rule["RAS_Oxygen_treatment_1.5vs21 hyb transcript"].loc[rxn] = sum(gene_numbers2)

                if "and" in str(df_gene_reaction_rule['gene_reaction_rule'].loc[rxn]):
                    df_gene_reaction_rule["RAS_Oxygen_treatment_1.5vs26 hyb transcript"].loc[rxn] = min(gene_numbers1)
                    df_gene_reaction_rule["RAS_Oxygen_treatment_1.5vs21 hyb transcript"].loc[rxn] = min(gene_numbers2)
                
            elif len(c) == 1:
                gene_numbers1 = []
                gene_numbers2 = []
                
                e = "SSO" + c[0]
                if e in df_oxygen_transcript.index:
                    gene_numbers1.append(df_oxygen_transcript["1.5vs26"].loc[e])
                    gene_numbers2.append(df_oxygen_transcript["1.5vs21"].loc[e])    
                        
                df_gene_reaction_rule["Oxygen_treatment_1.5vs26 hyb transcript"].loc[rxn] = gene_numbers1
                df_gene_reaction_rule["Oxygen_treatment_1.5vs21 hyb transcript"].loc[rxn] = gene_numbers2
                df_gene_reaction_rule["RAS_Oxygen_treatment_1.5vs26 hyb transcript"].loc[rxn] = min(gene_numbers1)
                df_gene_reaction_rule["RAS_Oxygen_treatment_1.5vs21 hyb transcript"].loc[rxn] = min(gene_numbers2)
        else:
            
            df_gene_reaction_rule["RAS_Oxygen_treatment_1.5vs26 hyb transcript"].loc[rxn] = 1000
            df_gene_reaction_rule["RAS_Oxygen_treatment_1.5vs21 hyb transcript"].loc[rxn] = 1000        
    elif rxn == "oxp_redox_1.6.5.3_RXN0__5330":
        g = df_gene_reaction_rule["genes"].loc[rxn].split("or")
        gene_numbers1 = []
        gene_numbers2 = []
        
        c1 = g[0].split(",")
        for i in range(len(c1)):
        
            e = "SSO" + c1[i]
            if e in df_oxygen_transcript.index:

                gene_numbers1.append(df_oxygen_transcript["1.5vs26"].loc[e])
                gene_numbers2.append(df_oxygen_transcript["1.5vs21"].loc[e]) 
#             if "or" in df_gene_reaction_rule['gene_reaction_rule'].loc[rxn]:
#                 df_gene_reaction_rule["RAS_Oxygen_treatment_1.5vs26 hyb transcript"].loc[rxn] = sum(gene_numbers1)
#                 df_gene_reaction_rule["RAS_Oxygen_treatment_1.5vs21 hyb transcript"].loc[rxn] = sum(gene_numbers2)

#             if "and" in str(df_gene_reaction_rule['gene_reaction_rule'].loc[rxn]):
#                 df_gene_reaction_rule["RAS_Oxygen_treatment_1.5vs26 hyb transcript"].loc[rxn] = min(gene_numbers1)
#                 df_gene_reaction_rule["RAS_Oxygen_treatment_1.5vs21 hyb transcript"].loc[rxn] = min(gene_numbers2)
        gene_numbers3 = []
        gene_numbers4 = []        
        c2 = g[1].split(",")
        for i in range(len(c2)):
            e = "SSO" + c2[i]
            if e in df_oxygen_transcript.index:
                gene_numbers3.append(df_oxygen_transcript["1.5vs26"].loc[e])
                gene_numbers4.append(df_oxygen_transcript["1.5vs21"].loc[e]) 

        df_gene_reaction_rule["RAS_Oxygen_treatment_1.5vs26 hyb transcript"].loc[rxn] = min(gene_numbers1) + min(gene_numbers2)
        df_gene_reaction_rule["RAS_Oxygen_treatment_1.5vs21 hyb transcript"].loc[rxn] = min(gene_numbers2) + min(gene_numbers4)
        

### Oxygen_treatment_1.5vs21 hyb transcript

In [ ]:
model1= M.copy()
df = pd.DataFrame()
df["soxM"] = ""
df["soxABCD"] = ""
df["doxABCDE"] = ""
df["soxM percent"] = ""
df["soxABCD percent"] = ""
df["doxABCDE percent"] = ""
for a in [1,3,5,7,9,11,13,15,100,1000]:
    for rxn in df_gene_reaction_rule.index: 
        if "." in str(df_gene_reaction_rule['RAS_Oxygen_treatment_1.5vs21 hyb transcript'].loc[rxn]):

            the_bound = a*df_gene_reaction_rule['RAS_Oxygen_treatment_1.5vs21 hyb transcript'].loc[rxn]

            if model.reactions.get_by_id(rxn).lower_bound < 0 and model1.reactions.get_by_id(rxn).upper_bound > 0: # this finds reversible rxns

                model1.reactions.get_by_id(rxn).bounds = (-1*the_bound, the_bound)# we set the bound(s) with max abs(FVA) 

            elif model.reactions.get_by_id(rxn).upper_bound > 0: # irreversible, flows in forward direction

                model1.reactions.get_by_id(rxn).bounds = (0, the_bound)  # the lower bound will stay zero

            elif model.reactions.get_by_id(rxn).lower_bound < 0: # irreversible, flows in backward direction

                model1.reactions.get_by_id(rxn).bounds = (-1*the_bound, 0) # the upper bound will stay zero
        else:
            if model.reactions.get_by_id(rxn).lower_bound < 0 and model1.reactions.get_by_id(rxn).upper_bound > 0: # this finds reversible rxns

                model1.reactions.get_by_id(rxn).bounds = (-1000, 1000)# we set the bound(s) with max abs(FVA) 

            if model.reactions.get_by_id(rxn).upper_bound > 0: # irreversible, flows in forward direction

                model1.reactions.get_by_id(rxn).bounds = (0, 1000)  # the lower bound will stay zero

            if model.reactions.get_by_id(rxn).lower_bound < 0: # irreversible, flows in backward direction

                model1.reactions.get_by_id(rxn).bounds = (-1000, 0) 
    model1.objective  = model1.reactions.NGAM
    sol = model1.optimize()
    df.loc[a] = [sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240'],sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240'],sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240'],sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240']/(sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240'] + sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240'] + sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']),sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240']/(sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']),sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']/(sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240'])]    

In [ ]:
df

In [ ]:
for rxn in df_gene_reaction_rule.index: 
    if "." in str(df_gene_reaction_rule['RAS_Oxygen_treatment_1.5vs26 hyb transcript'].loc[rxn]):

        the_bound = 100*df_gene_reaction_rule['RAS_Oxygen_treatment_1.5vs26 hyb transcript'].loc[rxn]

        if model.reactions.get_by_id(rxn).lower_bound < 0 and model1.reactions.get_by_id(rxn).upper_bound > 0: # this finds reversible rxns

            model1.reactions.get_by_id(rxn).bounds = (-1*the_bound, the_bound)# we set the bound(s) with max abs(FVA) 

        elif model.reactions.get_by_id(rxn).upper_bound > 0: # irreversible, flows in forward direction

            model1.reactions.get_by_id(rxn).bounds = (0, the_bound)  # the lower bound will stay zero

        elif model.reactions.get_by_id(rxn).lower_bound < 0: # irreversible, flows in backward direction

            model1.reactions.get_by_id(rxn).bounds = (-1*the_bound, 0) # the upper bound will stay zero
    else:
        if model.reactions.get_by_id(rxn).lower_bound < 0 and model1.reactions.get_by_id(rxn).upper_bound > 0: # this finds reversible rxns

            model1.reactions.get_by_id(rxn).bounds = (-1000, 1000)# we set the bound(s) with max abs(FVA) 

        if model.reactions.get_by_id(rxn).upper_bound > 0: # irreversible, flows in forward direction

            model1.reactions.get_by_id(rxn).bounds = (0, 1000)  # the lower bound will stay zero

        if model.reactions.get_by_id(rxn).lower_bound < 0: # irreversible, flows in backward direction

            model1.reactions.get_by_id(rxn).bounds = (-1000, 0) 
model1.objective  = model1.reactions.NGAM
sol = model1.optimize()
print ("NGAM", sol.fluxes.loc['NGAM'])
b = show_map(sol,map_loc)
b

### Oxygen_treatment_1.5vs26 hyb transcript

In [ ]:
model1= M.copy()
df = pd.DataFrame()
df["soxM"] = ""
df["soxABCD"] = ""
df["doxABCDE"] = ""
df["soxM percent"] = ""
df["soxABCD percent"] = ""
df["doxABCDE percent"] = ""
for a in [1,3,5,7,9,11,13,15,100,1000]:
    for rxn in df_gene_reaction_rule.index: 
        if "." in str(df_gene_reaction_rule['RAS_Oxygen_treatment_1.5vs26 hyb transcript'].loc[rxn]):

            the_bound = a*df_gene_reaction_rule['RAS_Oxygen_treatment_1.5vs26 hyb transcript'].loc[rxn]

            if model.reactions.get_by_id(rxn).lower_bound < 0 and model1.reactions.get_by_id(rxn).upper_bound > 0: # this finds reversible rxns

                model1.reactions.get_by_id(rxn).bounds = (-1*the_bound, the_bound)# we set the bound(s) with max abs(FVA) 

            elif model.reactions.get_by_id(rxn).upper_bound > 0: # irreversible, flows in forward direction

                model1.reactions.get_by_id(rxn).bounds = (0, the_bound)  # the lower bound will stay zero

            elif model.reactions.get_by_id(rxn).lower_bound < 0: # irreversible, flows in backward direction

                model1.reactions.get_by_id(rxn).bounds = (-1*the_bound, 0) # the upper bound will stay zero
        else:
            if model.reactions.get_by_id(rxn).lower_bound < 0 and model1.reactions.get_by_id(rxn).upper_bound > 0: # this finds reversible rxns

                model1.reactions.get_by_id(rxn).bounds = (-1000, 1000)# we set the bound(s) with max abs(FVA) 

            if model.reactions.get_by_id(rxn).upper_bound > 0: # irreversible, flows in forward direction

                model1.reactions.get_by_id(rxn).bounds = (0, 1000)  # the lower bound will stay zero

            if model.reactions.get_by_id(rxn).lower_bound < 0: # irreversible, flows in backward direction

                model1.reactions.get_by_id(rxn).bounds = (-1000, 0) 
    model1.objective  = model1.reactions.NGAM
    sol = model1.optimize()
    df.loc[a] = [sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240'],sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240'],sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240'],sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240']/(sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240'] + sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240'] + sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']),sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240']/(sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']),sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']/(sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240'])]    

In [ ]:
df

In [ ]:
# add tge gene reaction rule to the model
# reactions_ids = [rxn.id for rxn in model.reactions]
df_gene_reaction_rule["SSO_5 vs. -10"] = ""
df_gene_reaction_rule["RAS_5 vs. -10"] = ""

df_gene_reaction_rule["gene_reaction_relation"] = ""
for rxn in df_gene_reaction_rule.index:
    
    if rxn != "oxp_redox_1.6.5.3_RXN0__5330":
        if 'spontaneous' not in str(df_gene_reaction_rule['genes'].loc[rxn]):
            c = df_gene_reaction_rule["genes"].loc[rxn].split(",")
            if len(c) > 1:
                gene_rules = []
                gene_numbers1 = []
                gene_numbers2 = []
                
                for i in range(len(c)):
                    e = "SSO" + c[i]
                    if e in df_dynamic_shift.index: 
                        gene_rules.append(e)
#                         ge = df_gene_reaction_rule["SSO_5 vs. -10"].loc[e]
                       
                        gene_numbers1.append(df_dynamic_shift["5 vs.  -10"].loc[e])
                if "or" in df_gene_reaction_rule['gene_reaction_rule'].loc[rxn]:
                    df_gene_reaction_rule["RAS_5 vs. -10"].loc[rxn] = sum(gene_numbers1)

#                     df_gene_reaction_rule["RAS_control"].loc[rxn] = sum(gene_numbers2)
                if "and" in str(df_gene_reaction_rule['gene_reaction_rule'].loc[rxn]):
                    df_gene_reaction_rule["RAS_5 vs. -10"].loc[rxn] = min(gene_numbers1)

#                     df_gene_reaction_rule["RAS_control"].loc[rxn] = min(gene_numbers2)
                    

        
                d = str(" "+ str(df_gene_reaction_rule['gene_reaction_rule'].loc[rxn]) + " ").join(gene_rules)
                df_gene_reaction_rule["SSO_5 vs. -10"].loc[rxn] = gene_numbers1
#                 df_gene_reaction_rule["control"].loc[rxn] = gene_numbers2
                df_gene_reaction_rule["gene_reaction_relation"].loc[rxn] = "(" + d + ')'
            elif len(c) == 1:
                gene_rules = []
                gene_numbers1 = []
                
                e = "SSO" + c[0]
                if e in df_dynamic_shift.index:
                    d = e 
                    gene_numbers1.append(df_dynamic_shift["5 vs.  -10"].loc[e])                   
                    df_gene_reaction_rule["SSO_5 vs. -10"].loc[rxn] = gene_numbers1
                    df_gene_reaction_rule["gene_reaction_relation"].loc[rxn] = d
                    df_gene_reaction_rule["RAS_5 vs. -10"].loc[rxn] = df_dynamic_shift["5 vs.  -10"].loc[e]
        else:
            
            df_gene_reaction_rule["SSO_5 vs. -10"].loc[rxn] = 1000
                    
           
    elif rxn == "oxp_redox_1.6.5.3_RXN0__5330":
        g = df_gene_reaction_rule["genes"].loc[rxn].split("or")
        gene_rules = []
        gene_numbers1 = []
        gene_numbers2 = []
        c1 = g[0].split(",")
        for i in range(len(c1)):
        
            e = "SSO" + c1[i]
            if e in df_dynamic_shift.index:  
                gene_rules.append(e)
                gene_numbers1.append(df_dynamic_shift["5 vs.  -10"].loc[e])

        
               
        
        d1 = str(" "+ str(df_gene_reaction_rule['gene_reaction_rule'].loc[rxn]) + " ").join(gene_rules)
        gene_numbers3 = []
        gene_numbers4 = []
        c2 = g[1].split(",")
        for i in range(len(c2)):
        
            e = "SSO" + c2[i]
            if e in df_dynamic_shift.index:  
                gene_numbers3.append(df_dynamic_shift["5 vs.  -10"].loc[e])

               
        df_gene_reaction_rule["SSO_5 vs. -10"].loc[rxn] = gene_numbers1 + gene_numbers3

        d2 = str(" "+ str(df_gene_reaction_rule['gene_reaction_rule'].loc[rxn]) + " ").join(gene_rules)
        
        df_gene_reaction_rule["gene_reaction_relation"].loc[rxn] = "((" + d1 + ") or (" +  d2+ "))" 
        df_gene_reaction_rule["RAS_5 vs. -10"].loc[rxn] = min(gene_numbers1) + min(gene_numbers3)
#         df_gene_reaction_rule["RAS_control"].loc[rxn] = min(gene_numbers2) + min(gene_numbers4)


In [ ]:
df_gene_reaction_rule.to_csv('../data/oxygen_treatment2020.csv')

## The gearshifting among different electron transfer chain 

In [ ]:
model1 = model.copy()
model1.reactions.NGAM.bounds = (0,1000)
model1.objective = model1.reactions.NGAM
sol = model1.optimize()
l = sol.fluxes[abs(sol.fluxes)>1e-6].index
# for rxn in l:
#     print (rxn, sol.fluxes.loc[rxn])

for r in model1.reactions:
    if r.id in l:
#         print (r.id)
        if sol.fluxes.loc[r.id] > 0:
            model1.reactions.get_by_id(r.id).upper_bound = sol.fluxes.loc[r.id]
           
        elif sol.fluxes.loc[r.id] < 0:
            model1.reactions.get_by_id(r.id).lower_bound = sol.fluxes.loc[r.id]
          
    else:
        model1.reactions.get_by_id(r.id).bounds = (0,0)
b = show_map(sol,map_loc)
b

In [ ]:
df = pd.DataFrame()
df["soxM"] = ""
df["soxABCD"] = ""
df["doxABCDE"] = ""
df["soxM percent"] = ""
df["soxABCD percent"] = ""
df["doxABCDE percent"] = ""
for a in [1,3,5,7,9,11,13,15]:
    for rxn in df_gene_reaction_rule.index: 
        if "." in str(df_gene_reaction_rule['RAS_HU TREATED/CONTROL'].loc[rxn]):

            the_bound = a*df_gene_reaction_rule['RAS_HU TREATED/CONTROL'].loc[rxn]

            if model.reactions.get_by_id(rxn).lower_bound < 0 and model1.reactions.get_by_id(rxn).upper_bound > 0: # this finds reversible rxns

                model1.reactions.get_by_id(rxn).bounds = (-1*the_bound, the_bound)# we set the bound(s) with max abs(FVA) 

            elif model.reactions.get_by_id(rxn).upper_bound > 0: # irreversible, flows in forward direction

                model1.reactions.get_by_id(rxn).bounds = (0, the_bound)  # the lower bound will stay zero

            elif model.reactions.get_by_id(rxn).lower_bound < 0: # irreversible, flows in backward direction

                model1.reactions.get_by_id(rxn).bounds = (-1*the_bound, 0) # the upper bound will stay zero
        else:
            if model.reactions.get_by_id(rxn).lower_bound < 0 and model1.reactions.get_by_id(rxn).upper_bound > 0: # this finds reversible rxns

                model1.reactions.get_by_id(rxn).bounds = (-1000, 1000)# we set the bound(s) with max abs(FVA) 

            if model.reactions.get_by_id(rxn).upper_bound > 0: # irreversible, flows in forward direction

                model1.reactions.get_by_id(rxn).bounds = (0, 1000)  # the lower bound will stay zero

            if model.reactions.get_by_id(rxn).lower_bound < 0: # irreversible, flows in backward direction

                model1.reactions.get_by_id(rxn).bounds = (-1000, 0) 
    model1.objective  = model1.reactions.NGAM
    sol = model1.optimize()
    df.loc[a] = [sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240'],sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240'],sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240'],sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240']/(sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240'] + sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240'] + sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']),sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240']/(sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']),sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240']/(sol.fluxes.loc['oxp_redox_1.10.3.13_soxM_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_soxABCD_TRANS__RXN__240']+sol.fluxes.loc['oxp_redox_1.10.3.13_doxABCDE_TRANS__RXN__240'])]    
    
#     l = sol.fluxes[abs(sol.fluxes)>1e-6].index
#     for rxn in l:
#         print (rxn, sol.fluxes.loc[rxn])
#     # print ("NGAM", sol.fluxes.loc['NGAM'])
#     b = show_map(sol,map_loc)
#     b

In [ ]:
df

In [ ]:
model2 = model.copy()
for rxn in df_gene_reaction_rule.index:
    the_bound = df_gene_reaction_rule['RAS_control'].loc[rxn]/10

    if model.reactions.get_by_id(rxn).lower_bound < 0 and model.reactions.get_by_id(rxn).upper_bound > 0: # this finds reversible rxns
       
        model2.reactions.get_by_id(rxn).bounds = (-1*the_bound , the_bound) # we set the bound(s) with max abs(FVA) 
     
    elif model.reactions.get_by_id(rxn).upper_bound > 0: # irreversible, flows in forward direction
        
        model2.reactions.get_by_id(rxn).bounds = (0, the_bound)  # the lower bound will stay zero
      
    elif model.reactions.get_by_id(rxn).lower_bound < 0: # irreversible, flows in backward direction
        
        model2.reactions.get_by_id(rxn).bounds = (-1*the_bound, 0) # the upper bound will stay zero
        
      

    model2.repair()


In [ ]:
M = model2.copy()
M.objective  = M.reactions.NGAM
for rxn in M.reactions:
    if "T_flux_L" in rxn.id:
        rxn.bounds = (0,0)
sol = M.optimize()
l = sol.fluxes[abs(sol.fluxes)>1e-6].index
for rxn in l:   
    print (rxn,'\t',sol.fluxes.loc[rxn],'\t',model.reactions.get_by_id(rxn).reaction, '\n')
b = show_map(sol,map_loc)
b


In [ ]:
model1 = M.copy()

for rxn in df_gene_reaction_rule.index:
    df = pd.DataFrame(columns=["NGAM"])
    list = [1, 10, 100, 1000,10000]
    for a in list:
        
        the_bound = df_gene_reaction_rule['RAS_HU'].loc[rxn]/a

        if model1.reactions.get_by_id(rxn).lower_bound < 0 and model1.reactions.get_by_id(rxn).upper_bound > 0: # this finds reversible rxns

            model1.reactions.get_by_id(rxn).bounds = (-1*the_bound, the_bound) # we set the bound(s) with max abs(FVA) 

        elif model1.reactions.get_by_id(rxn).upper_bound > 0: # irreversible, flows in forward direction

            model1.reactions.get_by_id(rxn).bounds = (0, the_bound)  # the lower bound will stay zero

        elif int(model1.reactions.get_by_id(rxn).lower_bound) < 0: # irreversible, flows in backward direction

            model1.reactions.get_by_id(rxn).bounds = (-1*the_bound, 0) # the upper bound will stay zero
        model1.objective  = model1.reactions.NGAM
        sol = model1.optimize()
    df.loc[a] = [-sol.fluxes.loc["NGAM"]] 

In [ ]:
for rxn in df_gene_reaction_rule.index:        
    the_bound = df_gene_reaction_rule['RAS_control'].loc[rxn]/5

    if model1.reactions.get_by_id(rxn).lower_bound < 0 and model1.reactions.get_by_id(rxn).upper_bound > 0: # this finds reversible rxns

        model1.reactions.get_by_id(rxn).bounds = (-1*the_bound, the_bound) # we set the bound(s) with max abs(FVA) 

    elif model1.reactions.get_by_id(rxn).upper_bound > 0: # irreversible, flows in forward direction

        model1.reactions.get_by_id(rxn).bounds = (0, the_bound)  # the lower bound will stay zero

    elif int(model1.reactions.get_by_id(rxn).lower_bound) < 0: # irreversible, flows in backward direction

        model1.reactions.get_by_id(rxn).bounds = (-1*the_bound, 0) # the upper bound will stay zero
model1.objective  = model1.reactions.NGAM

sol = model1.optimize()
print ("NGAM",sol.fluxes.loc['NGAM'])
b = show_map(sol,map_loc)
b